# 1. การเตรียมข้อมูล (Preparing Data)

In [1]:
# import ไลบรารีที่จำเป็น
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# อนุญาตการเข้าถึง Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# กำหนดชื่อคอลัมน์
col_names = ['polarity', 'id', 'date', 'query', 'user', 'text']
# อ่านข้อมูลจากไฟล์ CSV โดยระบุ encoding เป็น latin1
data = pd.read_csv('/content/drive/My Drive/Dataset/training.1600000.processed.noemoticon.csv', encoding='latin1', names=col_names)
# ดูข้อมูล 10 แถวแรก
data.head(10)

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [4]:
#ลบคอลัมน์ที่ไม่จำเป็นออก
data = data.drop(columns=['id', 'date', 'query', 'user'])
data.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 1600000


In [6]:
#แสดงคอลัมน์ polarity
#0 คือค่า negative และ 4 คือค่า  positive
data.polarity.value_counts()

0    800000
4    800000
Name: polarity, dtype: int64

In [7]:
# แทนที่ค่า polarity เป็น 0 และ 1 ตามที่ระบุ 0 คือค่า negative และ 1 คือค่า  positive
data.polarity = data.polarity.replace({0: 0, 4: 1})
#แสดงคอลัมน์ polarity
data.polarity.value_counts()

0    800000
1    800000
Name: polarity, dtype: int64

In [8]:
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 1600000


In [9]:
#ลดจำนวนข้อมูลลงเพราะขนาดข้อมูลมีขนาดที่เยอะเกิน
data = data.sample(n=10000)
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 10000


In [10]:
#แสดงคอลัมน์ polarity
data.polarity.value_counts()

1    5038
0    4962
Name: polarity, dtype: int64

In [11]:
# ทำการลบข้อมูลซ้ำซ้อน
data = data.drop_duplicates()

In [12]:
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 9994


In [13]:
# คำนวณความยาวของข้อความแต่ละข้อความ
data['text_length'] = data['text'].apply(len)

In [14]:
# ทำความสะอาดข้อความโดยลบตัวอักษรที่ไม่เป็นตัวอักษรและแปลงเป็นตัวพิมพ์เล็กทั้งหมด
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

data['clean_text'] = data['text'].apply(clean_text)

In [15]:
# ตัดคำและลบ stopwords ออกจากข้อความ
stop_words = set(stopwords.words('english'))
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

data['tokens'] = data['clean_text'].apply(tokenize_and_remove_stopwords)

In [16]:
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 9994


In [17]:
# บันทึกข้อมูลที่ทำการปรับแต่งไว้
data.to_csv('/content/drive/My Drive/Dataset/cleaned_sentiment140.csv', index=False)

In [18]:
# เลือก feature (X) และ target (y)
X = data['tokens'].apply(lambda x: ' '.join(x))
y = data['polarity']

In [19]:
# แบ่งชุดข้อมูลเป็น Train set และ Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# สร้างเวกเตอร์คุณลักษณะโดยใช้ CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


# 2. การเลือกโมเดล (Model Selection)

In [21]:
# สร้างและประเมินโมเดล SVM
svm_model = SVC()
svm_scores = cross_val_score(svm_model, X_train_vectorized, y_train, cv=5)


In [22]:
# สร้างและประเมินโมเดล ANN
ann_model = MLPClassifier()
ann_scores = cross_val_score(ann_model, X_train_vectorized, y_train, cv=5)


In [23]:
# สร้างและประเมินโมเดล Naive Bayes
nb_model = MultinomialNB()
nb_scores = cross_val_score(nb_model, X_train_vectorized, y_train, cv=5)

In [24]:
# สร้างและประเมินโมเดล Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_scores = cross_val_score(lr_model, X_train_vectorized, y_train, cv=5)

In [25]:
# สร้างและประเมินโมเดล K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier()
knn_scores = cross_val_score(knn_model, X_train_vectorized, y_train, cv=5)

In [26]:
# แสดงผลคะแนน cross validation score ของแต่ละโมเดล
print("SVM Cross Validation Scores:", svm_scores)
print("ANN Cross Validation Scores:", ann_scores)
print("Naive Bayes Cross Validation Scores:", nb_scores)
print("Logistic Regression Cross Validation Scores:", lr_scores)
print("K-Nearest Neighbors (KNN) Cross Validation Scores:", knn_scores)

SVM Cross Validation Scores: [0.73233271 0.71482176 0.73045654 0.7191995  0.71106942]
ANN Cross Validation Scores: [0.67604753 0.64978111 0.70168856 0.69105691 0.67479675]
Naive Bayes Cross Validation Scores: [0.71794872 0.71544715 0.72983114 0.72920575 0.71169481]
Logistic Regression Cross Validation Scores: [0.72232645 0.70856785 0.73858662 0.71482176 0.71732333]
K-Nearest Neighbors (KNN) Cross Validation Scores: [0.60412758 0.62664165 0.61726079 0.61976235 0.64477799]


# 3. การประเมินโมเดล (Model Evaluation)

In [27]:
# import ไลบรารีที่จำเป็น
from sklearn.metrics import classification_report, confusion_matrix

####SVM

In [28]:
# ตัวอย่างการประเมินโมเดล SVM
svm_model.fit(X_train_vectorized, y_train)
svm_predictions = svm_model.predict(X_test_vectorized)

In [29]:
# แสดงค่า Classification Report
print("Classification Report for SVM:")
print(classification_report(y_test, svm_predictions))

Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       972
           1       0.72      0.77      0.74      1027

    accuracy                           0.73      1999
   macro avg       0.73      0.73      0.73      1999
weighted avg       0.73      0.73      0.73      1999



In [30]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for SVM:")
print(confusion_matrix(y_test, svm_predictions))

Confusion Matrix for SVM:
[[672 300]
 [241 786]]


####ANN

In [31]:
# ตัวอย่างการประเมินโมเดล ANN
ann_model.fit(X_train_vectorized, y_train)
ann_predictions = ann_model.predict(X_test_vectorized)

In [32]:
# แสดงค่า Classification Report
print("Classification Report for ANN:")
print(classification_report(y_test, ann_predictions))

Classification Report for ANN:
              precision    recall  f1-score   support

           0       0.65      0.67      0.66       972
           1       0.68      0.66      0.67      1027

    accuracy                           0.66      1999
   macro avg       0.66      0.66      0.66      1999
weighted avg       0.66      0.66      0.66      1999



In [33]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for ANN:")
print(confusion_matrix(y_test, ann_predictions))

Confusion Matrix for ANN:
[[649 323]
 [350 677]]


####Naive Bayes

In [34]:
# ตัวอย่างการประเมินโมเดล Naive Bayes
nb_model.fit(X_train_vectorized, y_train)
nb_predictions = nb_model.predict(X_test_vectorized)

In [35]:
# แสดงค่า Classification Report
print("Classification Report for Naive Bayes:")
print(classification_report(y_test, nb_predictions))

Classification Report for Naive Bayes:
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       972
           1       0.76      0.67      0.71      1027

    accuracy                           0.72      1999
   macro avg       0.72      0.72      0.72      1999
weighted avg       0.72      0.72      0.72      1999



In [36]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for Naive Bayes:")
print(confusion_matrix(y_test, nb_predictions))

Confusion Matrix for Naive Bayes:
[[749 223]
 [338 689]]


####Logistic Regression

In [37]:
# ตัวอย่างการประเมินโมเดล Logistic Regression
lr_model.fit(X_train_vectorized, y_train)
lr_predictions = lr_model.predict(X_test_vectorized)

In [38]:
# แสดงค่า Classification Report
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, lr_predictions))

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.72      0.72      0.72       972
           1       0.74      0.73      0.73      1027

    accuracy                           0.73      1999
   macro avg       0.73      0.73      0.73      1999
weighted avg       0.73      0.73      0.73      1999



In [39]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, lr_predictions))

Confusion Matrix for Logistic Regression:
[[701 271]
 [273 754]]


####K-Nearest Neighbors (KNN)

In [40]:
# ตัวอย่างการประเมินโมเดล K-Nearest Neighbors (KNN)
knn_model.fit(X_train_vectorized, y_train)
knn_predictions = knn_model.predict(X_test_vectorized)

In [41]:
# แสดงค่า Classification Report
print("Classification Report for K-Nearest Neighbors (KNN):")
print(classification_report(y_test, knn_predictions))

Classification Report for K-Nearest Neighbors (KNN):
              precision    recall  f1-score   support

           0       0.63      0.68      0.65       972
           1       0.67      0.62      0.64      1027

    accuracy                           0.65      1999
   macro avg       0.65      0.65      0.65      1999
weighted avg       0.65      0.65      0.65      1999



In [42]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for K-Nearest Neighbors (KNN):")
print(confusion_matrix(y_test, knn_predictions))

Confusion Matrix for K-Nearest Neighbors (KNN):
[[660 312]
 [395 632]]


# 4. การนำโมเดลไปใช้งาน (Deploy Model)

#### บันทึกโมเดลที่ดีกว่า

In [43]:
import pickle

# โมเดล SVM ที่ดีที่สุด
best_svm_model = svm_model

# บันทึกโมเดล SVM ด้วยโมดูล pickle ลงใน Google Drive
with open('/content/drive/My Drive/Dataset/best_svm_model.pkl', 'wb') as model_file:
    pickle.dump(best_svm_model, model_file)

#### ทดสอบโหลดโมเดลไปใช้งาน โดยการอ่านข้อมูลเข้ามา 1 ตัวอย่าง สกัดคุณลักษณะและทำนายผลลัพธ์

In [44]:
# โหลดโมเดล SVM ที่ถูกบันทึกไว้
loaded_model = pickle.load(open("/content/drive/My Drive/Dataset/best_svm_model.pkl", 'rb'))

In [45]:
# ข้อความที่ต้องการทำนาย
X_new_text = ["I hated the movie I watched last night. The plot was confusing, and the acting was terrible."] # Negative
Y_new_text = ["I absolutely loved the book I read recently. The characters were well-developed, and the story was captivating."] #Positive

In [46]:
# สกัดคุณลักษณะของข้อความเป็นตัวเลข
X_new_features = vectorizer.transform(X_new_text)
Y_new_features = vectorizer.transform(Y_new_text)

In [47]:
# ทำนายผลลัพธ์
predictionx = loaded_model.predict(X_new_features)
predictiony = loaded_model.predict(Y_new_features)

In [48]:
# แสดงผลลัพธ์
if predictiony[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

Positive Tweet
